In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("../podatki/knjige.csv")


In [3]:
df.head()



,naslov,avtor,ocena
0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,"4.35 avg rating — 9,717,517 ratings"
1,Pride and Prejudice,Jane Austen,"4.29 avg rating — 4,703,968 ratings"
2,To Kill a Mockingbird,Harper Lee,"4.26 avg rating — 6,764,408 ratings"
3,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,"4.50 avg rating — 3,733,285 ratings"
4,The Book Thief,Markus Zusak,"4.39 avg rating — 2,832,122 ratings"
